In [48]:
# This program is used to show the case of recommend location according to the input company.
# Each time a company is randomly picked out of the dataset and return several buildings accoring to their score

In [49]:
#describe each location with companies in side
import pandas as pd
import numpy as np
import os
import pygeohash as pgh
from math import *
from sklearn.metrics.pairwise import euclidean_distances
import random
import matplotlib.pyplot as plt
pjoin = os.path.join

In [50]:
#data load
datapath = '/Users/yefeichen/Database/location_recommender_system/'
cfile = ['dnb_pa.csv','dnb_sf.csv','dnb_sj.csv']
lfile = 'location_scorecard_190912.csv'
clfile = ['PA.csv','SF.csv','SJ.csv']
clscorefile = ['PA_comp_loc_score.csv','SF_comp_loc_score.csv','SJ_comp_loc_score.csv']

ind_city = 0

pdc = pd.read_csv(pjoin(datapath,cfile[ind_city]))
pdl = pd.read_csv(pjoin(datapath,lfile))
pdcl = pd.read_csv(pjoin(datapath,clfile[ind_city]))
pdclscore = pd.read_csv(pjoin(datapath,clscorefile[ind_city]))

In [51]:
#pick a company randomly
single_record = pdclscore.sample(n=1)
#find company info
company_info = single_record[['duns_number']].merge(pdc,on='duns_number',how='left',suffixes=['','_right'])
company_info.head()

,duns_number,business_name,latitude,longitude,global_ultimate_duns_number,global_ultimate_business_name,emp_here,emp_here_code,emp_here_range,emp_total,...,primary_sic_4_digit,primary_sic_4_digit_v2,primary_sic_2_digit,primary_sic_2_digit_v2,naics_6_digit,naics_6_digit_v2,naics_4_digit,naics_4_digit_v2,naics_2_digit,naics_2_digit_v2
0,36440553.0,The Leland Stanford Junior University,37.431727,-122.186149,9214214,Leland Stanford Junior University,48,Actual,30 - 49,0,...,8221,COLLEGES AND UNIVERSITIES,82,EDUCATIONAL SERVICES,611310,"Colleges, Universities, and Professional Schools",6113,"Colleges, Universities, and Professional Schools",61,Educational Services


In [52]:
#search for buildings
company_location_sim = pdclscore[pdclscore['duns_number']== company_info.loc[0,'duns_number']]
company_location_topK = company_location_sim.nsmallest(10,'sim_score').reset_index(drop=True)
company_location_topK

,duns_number,atlas_location_uuid,sim_score
0,36440553.0,0e7e05fa-1026-b0c5-4592-67608ae320b8,0.000000
1,36440553.0,a004dc42-6c3c-6f97-259d-7b2f10df424c,0.000000
2,36440553.0,91186755-a2d3-3920-7d49-b4bd91c58657,0.163620
3,36440553.0,b89c3096-5ebc-7491-2de8-79f22da62dbf,0.163620
4,36440553.0,6ba62017-ae70-43c3-afa0-cb4459fe51da,0.249017
5,36440553.0,c0e8517b-1fe0-b527-0f3f-41d4b56d6118,0.270814
6,36440553.0,0060ef47-b121-60b9-1983-02ebdb144dcf,0.382885
7,36440553.0,ee0733da-8dc3-c2a0-475a-2c1cf96c9ea6,0.382885
8,36440553.0,3083202a-936b-7d0e-f8b6-80d7ae73fa1a,0.482605
9,36440553.0,c5b270a7-6368-6e77-6039-618cc709f3a6,0.712630


In [53]:
#illustrate the building info
location_info = company_location_topK[['atlas_location_uuid','sim_score']].merge(pdl,on='atlas_location_uuid',how='left',suffixes=['','_right'])
location_info.head()

,atlas_location_uuid,sim_score,latitude,longitude,street_address,city,zip_code,state,score_predicted_eo,score_msa_percentile,...,pct_masters_degree,walk_score,bike_score,transit_score,model_version,score_predicted_eo_percentile,score_accessibility_percentile,score_demographics_percentile,score_employer_percentile,score_amenity_percentile
0,0e7e05fa-1026-b0c5-4592-67608ae320b8,0.000000,37.408572,-122.151483,3150 Porter Drive,Palo Alto,94304.0,CA,58.318389,32.579376,...,0.239327,29.0,72.0,NaN,us_1.0.0,0.561798,0.0,0.000000,45.505618,10.674157
1,a004dc42-6c3c-6f97-259d-7b2f10df424c,0.000000,37.436387,-122.178492,900 Welch Road,Palo Alto,94304.0,CA,52.886221,12.658050,...,0.217124,63.0,95.0,NaN,us_1.0.0,0.000000,0.0,42.696629,0.000000,10.674157
2,91186755-a2d3-3920-7d49-b4bd91c58657,0.163620,37.449481,-122.154334,705 Forest Avenue,Palo Alto,94301.0,CA,64.209835,65.664513,...,0.340345,78.0,84.0,NaN,us_1.0.0,5.056180,0.0,40.449438,46.629213,43.258427
3,b89c3096-5ebc-7491-2de8-79f22da62dbf,0.163620,37.433456,-122.128779,2741 Middlefield Road,Palo Alto,94306,CA,58.739942,35.501545,...,0.347636,65.0,95.0,NaN,us_1.0.0,0.561798,0.0,40.449438,20.786517,10.674157
4,6ba62017-ae70-43c3-afa0-cb4459fe51da,0.249017,37.425248,-122.136504,NaN,Palo Alto,94306.0,CA,70.190251,76.088789,...,0.138122,68.0,95.0,NaN,us_1.0.0,30.337079,0.0,12.921348,62.921348,10.674157
